# 概要

Twitterの位置情報をfoliumにマッピング出来るかなという思いつきから、<br>
Twitter APIを取り合えず触ってみることにした。<br>
例え、位置情報が取れなくても新たな知見になれば、それで良しとする。

##### 参考URL
* [公式Document](https://developer.twitter.com/en/docs)
* [公式和訳](http://westplain.sakuraweb.com/)
* [Qiita1](https://qiita.com/bakira/items/00743d10ec42993f85eb)
* [Qiita2](https://qiita.com/ogrew/items/0b267f57b8aaa24f1b73)
* [OAuth](https://ja.wikipedia.org/wiki/OAuth)

## 始め方

#### 1.開発者アカウントの登録
まず[ここ](https://developer.twitter.com/en/apps)からTwitter開発アカウントを登録する<br>
登録する際は、英語で200文字以上動機の記載等する必要がある<br>
私は英語いまいちなので、Google翻訳を使った。<br>
質問もいくつかあり回答必須。<br>
ただし、すぐに登録が承認されたので自動判定していると思う。<br>
何かあったら嫌なのできちんと答えておいた<br>

#### 2.開発するアプリケーションの登録

アプリケーション登録して<br>
Twitterにアクセスするためのキーとトークンを取得する必要がある。
* Consumer Key
* Consumer Secret
* Access Token(アプリケーション用のID)
* Access Token Secret(アプリケーション用のパス)

※上記はアップロード出来ないため、TwitterConfig.pyファイルに定義

#### 利用制限
* ツイートとリツイート（合算値）：3時間で300件
* いいね：24時間で1000件
* フォロー：24時間で1000件
* ダイレクトメッセージ：24時間で1万5000件

#### 提供APIとツール
* Standard APIs
* Premium APIs
* Enterprise APIs
* Ads APIs
* Twitter for websites<BR>
    
今回は無償のStandard APIsのみを使用<BR>
詳細は[ここ](https://developer.twitter.com/en/pricing)を参照
    
#### 3.開発

## Import

In [67]:
import json, TwitterConfig #標準のjsonモジュールとconfig.pyの読み込み
from requests_oauthlib import OAuth1Session #OAuthのライブラリの読み込み
#from IPython.core.debugger import Pdb; Pdb().set_trace()
CK = TwitterConfig.CONSUMER_KEY
CS = TwitterConfig.CONSUMER_SECRET
AT = TwitterConfig.ACCESS_TOKEN
ATS = TwitterConfig.ACCESS_TOKEN_SECRET
twitter = OAuth1Session(CK, CS, AT, ATS) #認証処理 数時間すると認証が切れるらしい

In [68]:
from enum import IntEnum
# TwitterAPI 結果コード
class TwtResCode(IntEnum):
    OK = 200,
    NotModified = 304,
    BadRequest = 400,
    Unauthorized = 401,
    Forbidden = 403,
    NotFound= 404,
    NotAcceptable=406,
    Gone = 410,
    EnhanceYourCalm = 420,
    UnprocessableEntity=422,
    TooManyRequests = 429,
    InternalServerError = 500,
    BadGateway = 502,
    ServiceUnavailable = 503,
    GatewayTimeout= 504

## Tweet

In [17]:
url = "https://api.twitter.com/1.1/statuses/update.json"

print("何をつぶやきますか?")
tweet = input('>> ')
print('----------------------------------------------------')

params = {"status" : tweet}

req = twitter.post(url, params = params)

if req.status_code == TwtResCode.OK:
    print("Succeed!")
else:
    print("ERROR : %d"% req.status_code)

何をつぶやきますか?
>> Test By TwitterAPI
----------------------------------------------------
Succeed!


## Favorites

In [63]:
url = "https://api.twitter.com/1.1/favorites/create.json"

print("フォボりたいTweetIDを入力してください")
tweet_id = input('>> ')
print('----------------------------------------------------')

params = {'id' : tweet_id}
req = twitter.post(url, params = params)
    
if req.status_code == TwtResCode.OK:
    print('https://twitter.com/Twitter/status/'+tweet_id+'を いいね しました')
else:
    print("ERROR : %d"% req.status_code)
    

フォボりたいTweetIDを入力してください
>> 1149407582488059909
----------------------------------------------------
https://twitter.com/Twitter/status/1149407582488059909を いいね しました


## Lookup(発話)

In [71]:
import pandas as pd
url = 'https://api.twitter.com/1.1/statuses/lookup.json'
TweetID = 1124983352187027457
res = twitter.get(url, params = {'id':TweetID})
if res.status_code != TwtResCode.OK:
    print("ERROR : %d"% res.status_code)
pd.set_option("display.max_colwidth", 200)
df = pd.read_json(res.text)
df.T

,0
contributors,NaN
coordinates,"{'type': 'Point', 'coordinates': [139.70053828, 35.69045138]}"
created_at,2019-05-05 10:25:06
entities,"{'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'JREast_official', 'name': 'JR東日本（公式）', 'id': 3104527237, 'id_str': '3104527237', 'indices': [16, 32]}], 'urls': [{'url': 'https:/..."
favorite_count,0
favorited,False
geo,"{'type': 'Point', 'coordinates': [35.69045138, 139.70053828]}"
id,1124983352187027457
id_str,1124983352187027456
in_reply_to_screen_name,NaN


位置情報を示していそうな項目はcoordinatesとgeoかな。
少し前にfolium使ったので折角なので寄り道してマッピングしてみる。

In [7]:
import folium
import ast
import re
tooltip = 'Click me!'
coordinates = re.findall(r'\[.+?\]', str(df.coordinates))[0]
m = folium.Map(location=[coordinates.split(',')[1][:-1],coordinates.split(',')[0][1:]],zoom_start=20)
folium.Marker(m.location, popup='<i>TweetID='+f"{TweetID}"+'</i>',tooltip=tooltip).add_to(m)
m

## Get Tweet timelines
クライアントアプリケーションが1回の要求で取得できるタイムラインの量には制限がある。<BR>
したがって、アプリケーションは、より完全なリストを作成するためにタイムラインの結果を反復処理する必要がある。<br>

In [18]:
#url = "https://api.twitter.com/1.1/statuses/user_timeline.json" #タイムライン取得エンドポイント
url = "https://api.twitter.com/1.1/statuses/mentions_timeline.json" #認証ユーザーに関する最新の20のTweet
#url = "https://api.twitter.com/1.1/statuses/home_timeline.json" #ホームページ最新の20のTweet

params ={'count' : 10} #取得数
res = twitter.get(url, params = params)

if res.status_code == TwtResCode.OK: #正常通信出来た場合
    timelines = json.loads(res.text) #レスポンスからタイムラインリストを取得
    for line in timelines: #タイムラインリストをループ処理
        print(line['user']['name']+'::'+line['text'])
        print(line['created_at'])
        print('*******************************************')
else: #正常通信出来なかった場合
    print("Failed: %d" % res.status_code)

NITORI::@pea47581220 ごめんなさい‼️今回はハズレです😭
しかし、
7/10まで毎日当たるチャンス🎊
またチャレンジしてくださいね♪

さらに、写真投稿キャンペーン同時開催中🎊
ニトリ商品券2000円分がもらえるWチャンス🎁… https://t.co/bfj9KZdaHa
Sat Jun 29 09:42:11 +0000 2019
*******************************************
NITORI::@pea47581220 ごめんなさい‼️今回はハズレです😭
しかし、
7/10まで毎日当たるチャンス🎊
またチャレンジしてくださいね♪

さらに、写真投稿キャンペーン同時開催中🎊
ニトリ商品券2000円分がもらえるWチャンス🎁… https://t.co/nByoXAdpdh
Sat Jun 29 09:41:20 +0000 2019
*******************************************
NITORI::@pea47581220 ごめんなさい‼️今回はハズレです😭
しかし、
7/10まで毎日当たるチャンス🎊
またチャレンジしてくださいね♪

さらに、写真投稿キャンペーン同時開催中🎊
ニトリ商品券2000円分がもらえるWチャンス🎁… https://t.co/TtsIBOHKso
Fri Jun 28 22:03:03 +0000 2019
*******************************************


## Search

In [19]:
url = "https://api.twitter.com/1.1/search/tweets.json"

print("何を調べますか?")
keyword = input('>> ')
print('----------------------------------------------------')


params = {'q' : keyword, 'count' : 5}

res = twitter.get(url, params = params)

if res.status_code == TwtResCode.OK:
    search_timeline = json.loads(res.text)
    for tweet in search_timeline['statuses']:
        print(tweet['user']['name'] + '::' + tweet['text'])
        print(tweet['created_at'])
        print('----------------------------------------------------')
else:
    print("ERROR: %d" % res.status_code)

何を調べますか?
>> Coursera
----------------------------------------------------
Javarevisited::RT @javinpaul: 10 of the Best Tensorflow Courses to Learn Machine Learning from Coursera and Udemy 
https://t.co/5dlg6byWQf

via DEV Commun…
Sat Jul 13 13:06:59 +0000 2019
----------------------------------------------------
Javarevisited::RT @javinpaul: "10 of the Best Tensorflow Courses to Learn Machine Learning from Coursera and Udemy" by @javinpaul #DEVcommunity https://t.…
Sat Jul 13 13:06:53 +0000 2019
----------------------------------------------------
Skarra::RT @HQawiyy: 16 Amazing Places to Learn Something New Every Day

1. CodeAcademy
2. Udemy
3. Khan Academy
4. Spreeder
5. Platzi
6. BigThink…
Sat Jul 13 13:05:34 +0000 2019
----------------------------------------------------
Melanie::@CynthiasParadox @coursera Thank you very much ❤️ it's this course: https://t.co/RuuVk3baOP
Sat Jul 13 13:03:12 +0000 2019
----------------------------------------------------
Manvi Kakkar::https://t.co/J6

## Filter
フォローしてるユーザーの投稿が自動で流れてくるやつ。<br>
実行に時間がかかるので省略する。ていうか終わらない？

In [13]:
#url = "https://stream.twitter.com/1.1/statuses/filter.json"
#res = twitter.post(url,data=dict(track="coursera"), stream=True)

## Get Trend/Place
各地のトレンドを取得できる。
地域の指定はYahooが定めたwoeidを指定<br>

#### 参考URL
[日本のwoeid](https://lab.syncer.jp/Document/Japanese-prefecture-woeid/)<br>
[woeidとは](https://en.wikipedia.org/wiki/WOEID)

In [65]:
import urllib3
import sys
url="https://api.twitter.com/1.1/trends/place.json"
params = {'id' : 1118370}
res = twitter.get(url, params = params)
if res.status_code == TwtResCode.OK:
    trend = json.loads(res.text)[0]
    for tweet in trend['trends']:
        print(tweet['name'])
else:
    print("ERROR: %d" % res.status_code)

Aqours
SEAMO
ココロオドル
中居くん
オールスター
腹筋太鼓
#nhkらじらー
アクア
#nitiasa
#precure
#リュウソウジャー
アナザークウガ
ウォズ
#ゲゲゲの鬼太郎
シューイチ
NEW電王
ミハルくん
ヘイセイバー
漢字75文字
MEGAMAX
加古川くん
プルンス
十五祭初日
タイムジャッカー
オーズの映画
海東さん
ビッグサイト
雨の日曜日
おはりょー
小川航基
Wアーマー
えれなさん
海東大樹
FLOW
#あなたは取り囲まれて生きている
#サンデーモーニング
#ジオウ
#がっちりマンデー
#RAS神戸DAY2
#日曜討論
#性格と特徴からあなたが何歳か当てる
#戸山香澄生誕祭2019
#shu1tv
#水溜りボンドANN0
#tge897


## Upload media

In [ ]:
url_media = "https://upload.twitter.com/1.1/media/upload.json"
url_text = "https://api.twitter.com/1.1/statuses/update.json"

print("添付画像の名前を入力(png形式のみ)")
media_name = input('>> ')
print('-----------------------------------')

files = {"media" : open(media_name+".png", 'rb')}
req_media = twitter.post(url_media, files = files)

if req_media.status_code != TwtResCode.OK:
    print("MEDIA UPLOAD FAILED... %s", req_media.text)
    exit()

media_id = json.loads(req_media.text)['media_id']
print("MEDIA ID: %d" % media_id)

print("何をつぶやきますか？")
tweet = input('>> ')
print('-----------------------------------')

params = {"status" : tweet, "media_ids" : [media_id]}
req_media = twitter.post(url_text, params = params)

if req_media.status_code != TwtResCode.OK:
    print("TEXT UPLOAD FAILED... %s", req_text.text)
    exit()

print("SUCCEED!")

#### 感想
割とコピペで動かない所もあった。APIの仕様が変わっているようだ。<br>
無償での利用だと制限がきついイメージ。<br>
仕事で使う際には各結果コードに対する振る舞いとかも調査する必要があると思うけど、Webエンジニアではないのでノウハウがない。
Pythonにも列挙体があることを初めて知った（マジックナンバー嫌いなのでうれしい）。